In [1]:
import os
import time
import requests
from requests.structures import CaseInsensitiveDict

### Ready for test

In [2]:
# # JSONファイルからデータをロード
# file_path = './Data/train.json'
# with open(file_path, 'r', encoding='utf-8') as file:
#     data = json.load(file)

# # Extracting the second line of dialogue from each conversation
# english_sentences = [item['conversation'][1]['en_sentence'] for item in data]
# japanese_sentences = [item['conversation'][1]['ja_sentence'] for item in data]

# # Writing the English sentences to a text file
# with open('./Data/english_sentences.txt', 'w', encoding='utf-8') as file:
#     file.write("\n".join(english_sentences))

# # Writing the Japanese sentences to a text file
# with open('./Data/japanese_sentences.txt', 'w', encoding='utf-8') as file:
#     file.write("\n".join(japanese_sentences))

## SAP Translate Hub

In [3]:
def translate_with_sap(file_path, access_token):
    url = "https://document-translation.api.us10.translationhub.cloud.sap/api/v1/translation?strictMode=false&targetLanguage=ja-JP&sourceLanguage=en-US"

    headers = CaseInsensitiveDict()
    headers["Authorization"] = f"Bearer {access_token}"
    headers["accept"] = "application/octet-stream"

    # 正しいMIMEタイプを指定
    mime_type = "text/plain"

    files = {
        'file': ('filename', open(file_path, 'rb'), mime_type)
    }

    response = requests.post(url, files=files, headers=headers)
    if response.status_code == 200:
        return response.content.decode('utf-8') #日本語訳の場合は「.decode('utf-8')」と付ける。
    else:
        return f"Translation Error: {response.status_code} {response.text}"

# トークンを取得
access_token = os.getenv('SAP_TRANSLATE_API')

# 翻訳を行う
result_sap = translate_with_sap("./Data/english_sentences.txt", access_token)
print(result_sap)

私の名前はH社の高市です。
その中から、全幹部にまつわる話題を選び、解説します。
よし この高田 進めてくれないか?
大丈夫、対面でも話したかったです。
私の名前は高田で、会社Iの広報IR事務所にいます。
いつもありがとう、私の名前は会社の高田です。
ああ、約束はないと思う。
どうもありがとうございました Trust Company Mのノーズです
どうしましたか?
弊社のカスタマーサービスにお電話いただき、ありがとうございます。
ゴトさん、お元気ですか。
休暇はどうでしたか。
皆の頑張りのおかげで、新しいソフトウェアの売上はすでに販売予測を上回っています。
岡野さんいるの?
米国B社が、取引する可能性があるとの発表を聞きましたか？
玉岡さん、お元気ですか。
私の名前はB社の増田さんで、先日電卓を注文しました。
こちらはノルディック・トラベル・マガジン担当編集長の出口さん。
お元気ですか。
山下さん。
会議を始めよう。
今日は川本さんですか？
今週の水曜日に中途採用の面接に参加すると聞きました。
こちらはB社の高田です。
私の名前は岡田さんで、今日は司会者になります。
ちょっといいか?
計画より少し早い?
どの店舗ですか?
当社は 50 周年を迎え、ブランドロゴの更新を決定しました。
本日のトピックは何ですか?
今年は何をするつもりですか。
これまで学習してきたものから、今までに当社の製品について十分な知識があるはずです。
ありがとう 自分で作りました
うまくいけば、インドネシアとつながる。
カーGは大人気モデルなので、値引きをするのは本当に大変でした。
ただうなずいて話しているのではなく、ここで聞くという言葉を言います。
あなたにとってメリットのある保険情報についてお話を伺います。
もう一度事態対処計画を練りたいと思います。
山本さん、状況説明をお願いします。
昨夜はカレーを作りすぎたので昼食にいくらか入れることにした。
食堂で定期的に食事をする人たちから集めたフィードバックを伝えるためにここにいる。
私はＮ社の成田です。ケータリングを注文します。
いや、卵だ。
なに、2ヶ月前も増えたんですよね。
皆でそれを実現するために、私たちに何ができるか考えてみたい。
さっそく取りかかりますので、目の前の資料を見てください。
はい 木下さんって何?
はい 河合です いつも 

## DeepL Translate

In [4]:
def translate_with_deepl(file_path, auth_key):
    url = "https://api-free.deepl.com/v2/document"

    headers = CaseInsensitiveDict()
    headers["Authorization"] = f"DeepL-Auth-Key {auth_key}"

    mime_type = "text/plain"
    file_name = os.path.basename(file_path)

    files = {
        'file': (file_name, open(file_path, 'rb'), mime_type)
    }
    data = {
        "target_lang": "JA",
        "source_lang": "EN"
    }

    response = requests.post(url, files=files, headers=headers, data=data)
    if response.status_code == 200:
        doc_id = response.json()['document_id']
        doc_key = response.json()['document_key']

        # 翻訳完了を待つ
        status_url = f"https://api-free.deepl.com/v2/document/{doc_id}"
        params = {"document_key": doc_key}
        while True:
            status_response = requests.get(status_url, headers=headers, params=params)
            if status_response.json()["status"] == "done":
                # 翻訳が完了したら、ダウンロードURLを取得
                download_url = f"https://api-free.deepl.com/v2/document/{doc_id}/result?document_key={doc_key}"
                # ここでダウンロードリクエストも認証ヘッダーを付与
                download_response = requests.get(download_url, headers=headers)
                if download_response.status_code == 200:
                    return download_response.content.decode('utf-8') #日本語訳の場合は「.decode('utf-8')」と付ける。
                else:
                    return f"Error fetching download URL: {download_response.status_code} {download_response.text}"
            elif status_response.json()["status"] == "error":
                return f"Error in translation process: {status_response.json().get('message', 'No error message available.')}"
            time.sleep(1)
    else:
        return f"Translation Error: {response.status_code} {response.text}"

# トークンを取得
auth_key = "aec6564c-6b33-4485-93fd-b822902cd0de:fx"

# 翻訳を行う
result_deepl = translate_with_deepl("./Data/english_sentences.txt", auth_key)
print(result_deepl)

H社の高市です。
その中で、幹部の皆さんに関係するテーマを選んで説明します。
では、高田さんからお願いできますか？
大丈夫です、私も直接お話ししたかったので。
I社広報IR室の高田と申します。
いつもありがとうございます、I社の高田と申します。
あ、アポは取っていないと思います。
いつもありがとうございます、信託M社の能勢と申します。
どうしました？
カスタマーサービスにお電話いただき、ありがとうございます。
後藤さん、お元気ですか。
お休みはいかがでしたか？
おかげさまで、新ソフトの売上が早くも予想を上回りました。
岡野さんはいらっしゃいますか？
取引するかもしれないアメリカのB社の発表、聞いた？
玉岡さん、お元気ですか。
先日電卓を注文したB社の増田と申します。
北欧旅行雑誌の担当編集者の出口です。
お元気ですか。
山下さん。
会議を始めます。
今日は川本さんはいらっしゃいますか？
今週の水曜日に中途採用の面接に参加されるそうですが。
B社の高田です。
本日司会を務めます岡田と申します。
ちょっといいですか？
あ、予定より少し早いですか？
どこの店舗ですか？
弊社は創立50周年ということで、ブランドロゴを一新することにしました。
今日のトピックは何ですか？
今年は何をしようかな？
今まで勉強してきたことから、もう当社製品に関する知識は十分でしょう。
ありがとうございます。
うまくいけば、インドネシアとのコネクションもできるだろう。
G号は人気車種ですから、値引きするのは本当に大変でした。
ここで「聞く」と言っても、ただうなずくだけの話ではない。
お客様にとって有益な保険情報についてお話ししたくてお電話しました。
不測の事態に備え、もう一度確認しておきたいと思います。
山本さん、状況を説明してください。
昨夜カレーを作りすぎたので、昼食に少し持ち込むことにしました。
いつも食堂で食事をしている人たちから集めた意見を伝えに来ました。
ケータリングを頼みたいのですが、N社の成田と申します。
いやあ、卵ですよ。
えっ、2ヶ月前も増えてましたよね？
そのために何ができるか、みんなで考えたいと思います。
早速ですが、目の前の資料を見てください。
はい、木下さん何ですか？
はい、河合です、いつもありがとうございます。
今日入社したばかりで、このあたりはあまり詳しくないのですが

### Compare

In [15]:
import nltk
from nltk.translate.bleu_score import sentence_bleu

# NLTKのデータダウンロード
nltk.download('punkt')

def load_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    return [line.strip() for line in lines]

from nltk.translate.bleu_score import SmoothingFunction

def calculate_scores_with_smoothing(translated_texts, reference_texts):
    bleu_scores = []
    smoothie = SmoothingFunction().method4

    for translated, reference in zip(translated_texts, reference_texts):
        reference_tokens = nltk.word_tokenize(reference)
        translated_tokens = nltk.word_tokenize(translated)
        bleu_score = sentence_bleu([reference_tokens], translated_tokens, smoothing_function=smoothie)

        bleu_scores.append(bleu_score)

    average_bleu = sum(bleu_scores) / len(bleu_scores)

    return average_bleu

# ファイルから正しい英文を読み込む
reference_texts = load_text("./Data/japanese_sentences.txt")

# 翻訳されたテキストをリストに変換
# result_sap_text = result_sap.decode('utf-8').split('\n')
# result_deepl_text = result_deepl.decode('utf-8').split('\n')

result_sap_text = result_sap.split('\n')
result_deepl_text = result_deepl.split('\n')

# 再計算の実行
sap_bleu_smooth = calculate_scores_with_smoothing(result_sap_text, reference_texts)
deepl_bleu_smooth = calculate_scores_with_smoothing(result_deepl_text, reference_texts)

print("SAP Translation (Smoothed) - BLEU Score:", sap_bleu_smooth)
print("DeepL Translation (Smoothed) - BLEU Score:", deepl_bleu_smooth)

[nltk_data] Downloading package punkt to /home/ryota/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


SAP Translation (Smoothed) - BLEU Score: 0.026702968121732788
DeepL Translation (Smoothed) - BLEU Score: 0.046511627906976744


In [28]:
import MeCab
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# MeCabの初期化
dic_path = "/var/lib/mecab/dic/ipadic"  # or use "/var/lib/mecab/dic/ipadic" if it contains the correct files
mecabrc_path = "/etc/mecabrc"
mecab = MeCab.Tagger(f'-d {dic_path} -r {mecabrc_path} -Owakati')


def tokenize_japanese(text):
    return mecab.parse(text).strip().split()

def load_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    return [line.strip() for line in lines]

def calculate_scores_with_smoothing_by_mecab(translated_texts, reference_texts):
    bleu_scores = []
    smoothie = SmoothingFunction().method4

    for translated, reference in zip(translated_texts, reference_texts):
        reference_tokens = tokenize_japanese(reference)
        translated_tokens = tokenize_japanese(translated)
        bleu_score = sentence_bleu([reference_tokens], translated_tokens, smoothing_function=smoothie)
        bleu_scores.append(bleu_score)

    average_bleu = sum(bleu_scores) / len(bleu_scores)
    return average_bleu

# BLEUスコアの計算
sap_bleu_smooth = calculate_scores_with_smoothing_by_mecab(result_sap_text, reference_texts)
deepl_bleu_smooth = calculate_scores_with_smoothing_by_mecab(result_deepl_text, reference_texts)

print("SAP Translation (Smoothed) - BLEU Score:", sap_bleu_smooth)
print("DeepL Translation (Smoothed) - BLEU Score:", deepl_bleu_smooth)

SAP Translation (Smoothed) - BLEU Score: 0.12562392844633447
DeepL Translation (Smoothed) - BLEU Score: 0.21407397018397395
